# **1. Libraries**

## **1.1 Requirements:**

In [66]:
%pip install pandas sentence-transformers requests google-generativeai openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


## **1.2. Imports:**

In [67]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import requests
import json
import google.generativeai as genai
import openai
import warnings
warnings.filterwarnings('ignore')

# **2. Dataset**

In [68]:
def load_motorcycle_dataset():
    try:
        print("Cargando dataset de motocicletas desde archivo local...")
        df = pd.read_csv("data/all_bikez_curated.csv")  
        print(f"Dataset cargado: {len(df)} registros")
        return df
    except Exception as e:
        print(f"Error cargando dataset local: {e}")

df = load_motorcycle_dataset()

Cargando dataset de motocicletas desde archivo local...
Dataset cargado: 38472 registros


In [69]:
df.head()   

,Brand,Model,Year,Category,Rating,Displacement (ccm),Power (hp),Torque (Nm),Engine cylinder,Engine stroke,...,Dry weight (kg),Wheelbase (mm),Seat height (mm),Front brakes,Rear brakes,Front tire,Rear tire,Front suspension,Rear suspension,Color options
0,acabion,da vinci 650-vi,2011,Prototype / concept model,3.2,NaN,804.0,NaN,Electric,Electric,...,420.0,NaN,NaN,Single disc,Single disc,NaN,NaN,NaN,NaN,NaN
1,acabion,gtbo 55,2007,Sport,2.6,1300.0,541.0,420.0,In-line four,four-stroke,...,360.0,NaN,NaN,Bajaj,NaN,NaN,NaN,NaN,NaN,NaN
2,acabion,gtbo 600 daytona-vi,2011,Prototype / concept model,3.5,NaN,536.0,NaN,Electric,Electric,...,420.0,NaN,NaN,Single disc,Single disc,NaN,NaN,NaN,NaN,NaN
3,acabion,gtbo 600 daytona-vi,2021,Prototype / concept model,NaN,NaN,536.0,NaN,Electric,Electric,...,420.0,NaN,NaN,Single disc,Single disc,NaN,NaN,NaN,NaN,NaN
4,acabion,gtbo 70,2007,Prototype / concept model,3.1,1300.0,689.0,490.0,In-line four,four-stroke,...,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Custom made.


# **3. Embeddings**

In [70]:
df['full_name'] = df['Brand'].astype(str) + ' ' + df['Model'].astype(str) + df['Year'].astype(str) + df['Category'].astype(str)

model = SentenceTransformer('all-MiniLM-L6-v2')
bike_embeddings = model.encode(df['full_name'].tolist(), convert_to_tensor=True)

print(f"Embeddings generados para {len(df)} motos")

Embeddings generados para 38472 motos


In [71]:
bike_embeddings

tensor([[-1.2322e-01, -1.4466e-02, -3.9557e-02,  ..., -4.5319e-02,
         -2.6322e-03,  1.2285e-02],
        [-1.4250e-01,  2.7370e-02, -1.1882e-01,  ...,  1.9495e-02,
         -8.0331e-02, -3.8742e-02],
        [-1.1961e-01, -5.2001e-03, -8.2352e-02,  ..., -3.2865e-02,
          4.5588e-06,  1.4345e-02],
        ...,
        [-1.2165e-01,  9.0539e-02, -1.3357e-01,  ...,  5.6523e-02,
         -6.6423e-02, -2.2853e-02],
        [-1.5018e-01,  9.5444e-02, -1.4290e-01,  ...,  4.4848e-02,
         -5.5807e-02, -3.3117e-02],
        [-1.5029e-01,  1.0674e-01, -1.3378e-01,  ...,  5.0834e-02,
         -6.6049e-02, -3.6776e-02]], device='mps:0')

# **4. Models**

In [92]:
def preparar_consulta(consulta, threshold=0.6, top_n=100):
    print(f"Procesando consulta: '{consulta}'")

    query_embedding = model.encode(consulta, convert_to_tensor=True)
    scores = util.cos_sim(query_embedding, bike_embeddings)[0]
    best_idx = scores.argmax().item()
    best_score = scores[best_idx].item()

    print(f"Mejor coincidencia embedding: {best_score:.3f}")
    if best_score >= threshold:
        print(f"   -> {df.iloc[best_idx]['Brand']} {df.iloc[best_idx]['Model']}")

    # Seleccionar las top-N motos más similares
    top_indices = scores.topk(top_n).indices.tolist()

    catalogo_str = ""
    for i, idx in enumerate(top_indices):
        row = df.iloc[idx]
        catalogo_str += f"{i+1}. {row['Brand']} {row['Model']}"
        if 'Displacement (ccm)' in row and pd.notna(row['Displacement (ccm)']):
            catalogo_str += f" {int(row['Displacement (ccm)'])}cc"
        if 'Year' in row and pd.notna(row['Year']):
            catalogo_str += f" ({int(row['Year'])})"
        if 'Category' in row and pd.notna(row['Category']):
            catalogo_str += f" - {row['Category']}"
        catalogo_str += "\n"

    return catalogo_str, best_score


In [93]:
def generar_prompt(consulta, catalogo_str):
    return (
        "Eres experto en motocicletas. Identifica la moto exacta basándote en la consulta no estructurada.\n\n"
        f"{catalogo_str}\n"
        f'CONSULTA: "{consulta}"\n\n'
        "REGLAS:\n"
        '- "BAJ" o "BAJA" = Bajaj\n'
        '- "DISC" o "DISCO" = Discover\n'
        "- Números pueden ser cilindraje o año\n"
        "- Corrige errores de escritura\n"
        "- Elige la coincidencia más probable\n\n"
        "RESPONDE SOLO JSON VÁLIDO SIN MARKDOWN:\n"
        "{{\n"
        '  "marca": "Marca exacta",\n'
        '  "modelo": "Modelo exacto",\n'
        '  "cilindraje": "XXX o null",\n'
        '  "año": "XXXX o null",\n'
        '  "categoria": "Categoría o null",\n'
        '  "confianza": 0.95,\n'
        '  "razonamiento": "Explicación breve"\n'
        "}}"
    )


In [74]:
def limpiar_respuesta_json(respuesta_texto):

    if "```json" in respuesta_texto:
        respuesta_texto = respuesta_texto.split("```json")[1].split("```")[0].strip()
    elif "```" in respuesta_texto:
        respuesta_texto = respuesta_texto.split("```")[1].strip()
    try:
        return json.loads(respuesta_texto)
    except json.JSONDecodeError:
        import re
        json_match = re.search(r'\{.*?\}', respuesta_texto, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())
            except json.JSONDecodeError:
                pass
        
        return {"error": "Respuesta no es JSON válido", "raw": respuesta_texto}

## **4.1. DeepSeek:**

In [94]:
class DeepSeekAdapter:
    def __init__(self, api_key):
        self.api_key = api_key
        self.name = "DeepSeek"
    
    def llamar_llm(self, prompt):
        try:
            print(f"Llamando a DeepSeek con prompt: {prompt}")  
            response = requests.post(
                "https://api.deepseek.com/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": "deepseek-chat",
                    "messages": [{"role": "user", "content": prompt}],
                    "max_tokens": 300,
                    "temperature": 0.1
                },
                timeout=30
            )
            
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                return {"error": f"API Error {response.status_code}"}
                
        except Exception as e:
            return {"error": f"Error conexión DeepSeek: {str(e)}"}

## **4.2. Gemini:**

In [130]:
class GeminiAdapter:
    def __init__(self, api_key, model_name="gemini-2.0-flash"):
        self.api_key = api_key
        self.model_name = model_name
        self.name = "Gemini"
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name)
    
    def llamar_llm(self, prompt):
        try:
            generation_config = genai.types.GenerationConfig(
                temperature=0.1,
                max_output_tokens=300,
            )
            
            response = self.model.generate_content(
                prompt,
                generation_config=generation_config
            )
            
            return response.text
            
        except Exception as e:
            return {"error": f"Error conexión Gemini: {str(e)}"}

# 5. Testing

In [ ]:
API_KEYS = {
    "deepseek": "xxxx-xxxx-xxxx-xxxx-xxxx",
    "gemini": "xxxx-xxxx-xxxx-xxxx-xxxx", 
}

In [78]:
def buscar_moto_universal(consulta, llm_adapter):
    catalogo_str, best_score = preparar_consulta(consulta)
    
    if best_score < 0.3:
        return {"error": "Consulta no coincide con ninguna moto en el catálogo"}
    
    prompt = generar_prompt(consulta, catalogo_str)
    
    print(f"Enviando prompt al modelo {llm_adapter.name}...")
    
    respuesta_texto = llm_adapter.llamar_llm(prompt)
    
    if isinstance(respuesta_texto, dict) and "error" in respuesta_texto:
        return respuesta_texto
    
    print("Respuesta recibida del modelo.")
    
    resultado_json = limpiar_respuesta_json(respuesta_texto)
    
    if "error" in resultado_json:
        return resultado_json
    
    return resultado_json

In [ ]:
def ejecutar_casos_prueba_universal(llm_adapter):
    casos_prueba = [
        {"input": "BAJ DISC 125 2015", "esperado": "Bajaj Discover 125 2015"},
        {"input": "DISCO 125", "esperado": "Bajaj Discover 125"},
        {"input": "DISCOVER 126", "esperado": "Bajaj Discover 126"},
        {"input": "125ST 15'", "esperado": "Motocicleta 125cc del 2015"},
        {"input": "FAZER 125", "esperado": "Yamaha Fazer 125"},
        {"input": "FZ 125", "esperado": "Yamaha FZ 125"},
        {"input": "YAMAHA FZ z", "esperado": "Yamaha FZ 125"},
    ]
    
    print(f"INICIANDO PRUEBAS CON {llm_adapter.name}")
    print("=" * 60)
    
    resultados = []
    
    for i, caso in enumerate(casos_prueba, 1):
        print(f"\n[{i}/{len(casos_prueba)}] PROCESANDO CASO")
        print("=" * 50)
        print(f"INPUT: '{caso['input']}'")
        print(f"ESPERADO: {caso['esperado']}")
        print("-" * 30)
        
        try:
            resultado = buscar_moto_universal(caso['input'], llm_adapter)
            
            if "error" not in resultado:
                print("RESULTADO EXITOSO:")
                print(f"  Marca: {resultado.get('marca', 'N/A')}")
                print(f"  Modelo: {resultado.get('modelo', 'N/A')}")
                print(f"  Cilindraje: {resultado.get('cilindraje', 'N/A')}")
                print(f"  Año: {resultado.get('año', 'N/A')}")
                print(f"  Categoría: {resultado.get('categoria', 'N/A')}")
                print(f"  Confianza: {resultado.get('confianza', 0):.2f}")
                print(f"  Razonamiento: {resultado.get('razonamiento', 'N/A')}")
            else:
                print(f"ERROR: {resultado['error']}")
                if 'raw' in resultado:
                    print(f"Respuesta cruda: {resultado['raw'][:200]}...")
            
            resultados.append({
                'caso_num': i,
                'input': caso['input'],
                'esperado': caso['esperado'],
                'resultado': resultado,
                'modelo': llm_adapter.name,
                'timestamp': pd.Timestamp.now()
            })
            
        except Exception as e:
            print(f"EXCEPCIÓN: {str(e)}")
            resultados.append({
                'caso_num': i,
                'input': caso['input'],
                'esperado': caso['esperado'],
                'resultado': {'error': f'Excepción: {str(e)}'},
                'modelo': llm_adapter.name,
                'timestamp': pd.Timestamp.now()
            })
        
        print("-" * 50)
    
    return resultados

In [127]:
def comparar_modelos(modelos_disponibles):
    todos_resultados = {}
    
    for nombre, adapter in modelos_disponibles.items():
        if adapter is not None:
            print(f"\n{'='*80}")
            print(f"PROBANDO MODELO: {nombre.upper()}")
            print(f"{'='*80}")
            
            resultados = ejecutar_casos_prueba_universal(adapter)
            todos_resultados[nombre] = resultados
        else:
            print(f"\nSaltando {nombre} - API key no configurada")
    
    return todos_resultados

In [131]:
modelos_disponibles = {}    
# DeepSeek
# modelos_disponibles["deepseek"] = DeepSeekAdapter(API_KEYS["deepseek"])
# Gemini
modelos_disponibles["gemini"] = GeminiAdapter(API_KEYS["gemini"])
    
print("Modelos disponibles:", list(modelos_disponibles.keys()))
    
# Opción 1: Probar un modelo específico
# if "deepseek" in modelos_disponibles:
#     resultados_deepseek = ejecutar_casos_prueba_universal(modelos_disponibles["deepseek"])

if "gemini" in modelos_disponibles:
    print("\nProbando solo Gemini:")
    resultados_gemini = ejecutar_casos_prueba_universal(modelos_disponibles["gemini"])

# Opción 2: Comparar múltiples modelos
# print("\nComparando todos los modelos disponibles:")
# resultados_comparacion = comparar_modelos(modelos_disponibles)


Modelos disponibles: ['gemini']

Probando solo Gemini:
INICIANDO PRUEBAS CON Gemini

[1/1] PROCESANDO CASO
INPUT: 'YAMAHA Fazer'
ESPERADO: Yamaha FZ 125
------------------------------
Procesando consulta: 'YAMAHA Fazer'
Mejor coincidencia embedding: 0.902
   -> yamaha fazer
Enviando prompt al modelo Gemini...
Respuesta recibida del modelo.
RESULTADO EXITOSO:
  Marca: Yamaha
  Modelo: Fazer
  Cilindraje: None
  Año: None
  Categoría: None
  Confianza: 0.80
  Razonamiento: La consulta especifica 'Yamaha Fazer' sin más detalles.  Hay múltiples modelos Fazer con diferentes cilindrajes y años.  Por lo tanto, no se puede determinar un modelo específico.
--------------------------------------------------


In [139]:
def exportar_resultados(resultados, archivo="salida.txt"):
    with open(archivo, "w", encoding="utf-8") as f:
        for res in resultados:
            r = res["resultado"]
            f.write("=" * 60 + "\n")
            f.write(f"Caso #{res['caso_num']} — Modelo: {res['modelo']}\n")
            f.write(f"Entrada: {res['input']}\n")
            f.write(f"Esperado: {res['esperado']}\n\n")

            if "error" in r:
                f.write(f"ERROR: {r['error']}\n")
                if 'raw' in r:
                    f.write(f"Respuesta cruda: {r['raw'][:200]}...\n")
            else:
                f.write("RESULTADO DEL MODELO:\n\n")
                f.write("┌────────────────┬──────────────────────────────┐\n")
                f.write(f"│ {'Marca':<16}│ {r.get('marca', 'N/A'):<28}│\n")
                f.write(f"│ {'Modelo':<16}│ {r.get('modelo', 'N/A'):<28}│\n")
                f.write(f"│ {'Cilindraje':<16}│ {r.get('cilindraje', 'N/A'):<28}│\n")
                f.write(f"│ {'Año':<16}│ {str(r.get('año', 'N/A')):<28}│\n")
                f.write(f"│ {'Categoría':<16}│ {str(r.get('categoria', 'N/A')):<28}│\n")
                f.write(f"│ {'Confianza':<16}│ {r.get('confianza', 0):<28.2f}│\n")
                f.write("└────────────────┴──────────────────────────────┘\n\n")
                f.write("Razonamiento:\n" + r.get('razonamiento', 'N/A') + "\n")
            f.write("=" * 60 + "\n\n")

    print(f"\nResultados exportados a '{archivo}'")


In [140]:
exportar_resultados(resultados_gemini)

TypeError: unsupported format string passed to NoneType.__format__

In [141]:
resultados_gemini[0]

{'caso_num': 1,
 'input': 'YAMAHA Fazer',
 'esperado': 'Yamaha FZ 125',
 'resultado': {'marca': 'Yamaha',
  'modelo': 'Fazer',
  'cilindraje': None,
  'año': None,
  'categoria': None,
  'confianza': 0.8,
  'razonamiento': "La consulta especifica 'Yamaha Fazer' sin más detalles.  Hay múltiples modelos Fazer con diferentes cilindrajes y años.  Por lo tanto, no se puede determinar un modelo específico."},
 'modelo': 'Gemini',
 'timestamp': Timestamp('2025-07-28 07:52:35.029111')}